In [2]:
# import prerequesite for blob
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobClient
from utils.blob import list_blob_in_container,get_blob_infos,download_blob
# import prerequesite for ai
import json
import requests
import logging
from utils.ai import is_ai_ready,get_prediction,get_json_prediction,get_clean_timed_prediction,get_trash_label,map_label_to_trash_id_PG,get_trash_first_time,get_trash_time_index,get_df_prediction,get_df_manual_trash
# import prerequesite for gps
import gpxpy
import gpxpy.gpx
import json
import subprocess
import datetime
from datetime import datetime
from datetime import timedelta
from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform
from tqdm import tqdm
from utils.gps import extract_gpx_from_gopro,parse_gpx,get_gps_point_list,create_time,create_latitude,create_longitude,create_elevation,fill_gps,long_lat_to_shape_point,transform_geo,get_df_trash_gps,get_gpx_name,get_df_manual_gps
# import prerequesite from media
from utils.media import get_media_duration,get_media_fps
# import prerequesite for postgre
import os
import psycopg2
from utils.postgre import get_pg_connection_string,open_pg_connection,close_pg_connection,insert_trash_2,insert_trash_df,get_df_data
# import exception
from utils.exceptions import ETLError
import warnings
warnings.filterwarnings('ignore')
# import argparse to pass parameters to main function
import argparse
import pathlib

In [3]:
with open('../datasample/prediction.json') as json_file:
    jsonPrediction = json.load(json_file)

In [4]:
DOWNLOAD_PATH = '/tmp'
AI_PORT = '5000'

In [5]:
blob_name = 'mobile.mp4'
container_name = 'campaign0'
source_data = 'mobile'
prediction_source = 'json'

connection_string = os.getenv("CONN_STRING")
pgserver = os.getenv("PGSERVER")
pgdatabase = os.getenv("PGDATABASE")
pgusername = os.getenv("PGUSERNAME")
pgpassword = os.getenv("PGPWD")

In [6]:
if not blob_name in os.listdir(DOWNLOAD_PATH):
    blob_video_client = BlobClient.from_connection_string(
        conn_str=connection_string, container_name=container_name, blob_name=blob_name)
    download_blob(blob_video_client, DOWNLOAD_PATH)
if source_data == 'mobile':
    blob_gpx_name = get_gpx_name(blob_name)
    if not blob_gpx_name in os.listdir(DOWNLOAD_PATH):
        blob_gpx_client = BlobClient.from_connection_string(
            conn_str=connection_string, container_name=container_name, blob_name=blob_gpx_name)
        download_blob(blob_gpx_client, DOWNLOAD_PATH)

In [7]:
if source_data == 'gopro' or source_data == 'mobile':
            # From AI inference server
            if prediction_source == 'ai':
                # Test that AI service is ready
                ai_ready = is_ai_ready(f'{ai_url}:{AI_PORT}')
                if ai_ready == True:
                    prediction = get_prediction(
                        blob_name, f'{ai_url}:{AI_PORT}')
                else:
                    logger.info(
                        "Early exit of ETL workflow as AI service is not available")
                    exit()
                # Cast prediction to JSON/Dictionnary format
                json_prediction = get_json_prediction(prediction)

            # From JSON file
            elif prediction_source == 'json':
                with open('../datasample/prediction.json') as json_file:
                    json_prediction = json.load(json_file)

In [8]:
if source_data == 'gopro':
    gpx_path = extract_gpx_from_gopro(f'{DOWNLOAD_PATH}/{blob_name}')
elif source_data == 'mobile' or source_data == 'manual':
    gpx_path = f'{DOWNLOAD_PATH}/{blob_gpx_name}'

# GPX parsing
gpx_data = parse_gpx(gpx_path)

In [9]:
# GPS Points
if source_data == 'gopro' or source_data == 'mobile':
    gps_points = get_gps_point_list(gpx_data)  # full list of gps point
elif source_data == 'manual':
    gps_points = gpx_data.waypoints  # list of gps point with trash

In [10]:
# Video duration
if source_data == 'gopro' or source_data == 'mobile':
    video_duration = get_media_duration(f'{DOWNLOAD_PATH}/{blob_name}')
    media_fps = get_media_fps(json_prediction)

if source_data == 'gopro' or source_data == 'mobile':
    video_duration_sup = int(video_duration)+1
    gps_points_filled = fill_gps(gps_points, video_duration_sup)


In [11]:
if source_data == 'gopro' or source_data == 'mobile':
    df_predictions = get_df_prediction(json_prediction, media_fps)
    df_trash_gps = get_df_trash_gps(df_predictions, gps_points_filled)
    df_data = get_df_data(df_predictions, df_trash_gps)
    campaign_id = 'ec501e35-b022-4c73-9988-a41218d6105e'
    df_data['campaign_id'] = campaign_id

In [14]:
df_data

,time_index,frame_to_box,id,label,trash_type_id,Time,the_geom,Latitude,Longitude,Elevation,campaign_id
0,0,"[0.27, 0.12, 0.3, 0.18]",0,fragments,4,2020-02-28 11:20:34+00:00,POINT (340207.587546391 6288394.037647896),43.603185,-1.457192,59.616760,ec501e35-b022-4c73-9988-a41218d6105e
1,0,"[0.28, 0.45, 0.29, 0.48]",1,fragments,4,2020-02-28 11:20:34+00:00,POINT (340207.587546391 6288394.037647896),43.603185,-1.457192,59.616760,ec501e35-b022-4c73-9988-a41218d6105e
2,0,"[0.18, 0.43, 0.2, 0.48]",2,fragments,4,2020-02-28 11:20:34+00:00,POINT (340207.587546391 6288394.037647896),43.603185,-1.457192,59.616760,ec501e35-b022-4c73-9988-a41218d6105e
3,1,"[0.15, 0.15, 0.18, 0.21]",3,fragments,4,2020-02-28 11:20:35+00:00,POINT (340207.8647825579 6288393.976342013),43.603184,-1.457189,57.599792,ec501e35-b022-4c73-9988-a41218d6105e
4,1,"[0.32, 0.11, 0.35, 0.17]",4,fragments,4,2020-02-28 11:20:35+00:00,POINT (340207.8647825579 6288393.976342013),43.603184,-1.457189,57.599792,ec501e35-b022-4c73-9988-a41218d6105e
5,1,"[0.17, 0.16, 0.19, 0.23]",5,fragments,4,2020-02-28 11:20:35+00:00,POINT (340207.8647825579 6288393.976342013),43.603184,-1.457189,57.599792,ec501e35-b022-4c73-9988-a41218d6105e
6,2,"[0.42, 0.38, 0.45, 0.51]",6,fragments,4,2020-02-28 11:20:36+00:00,POINT (340208.0997494296 6288395.099614676),43.603194,-1.457187,57.375122,ec501e35-b022-4c73-9988-a41218d6105e
7,2,"[0.99, 0.35, 1.0, 0.4]",7,fragments,4,2020-02-28 11:20:36+00:00,POINT (340208.0997494296 6288395.099614676),43.603194,-1.457187,57.375122,ec501e35-b022-4c73-9988-a41218d6105e
8,3,"[0.28, 0.4, 0.31, 0.51]",8,fragments,4,2020-02-28 11:20:37+00:00,POINT (340208.1528626664 6288395.023143975),43.603194,-1.457186,56.295898,ec501e35-b022-4c73-9988-a41218d6105e
9,3,"[0.19, 0.14, 0.23, 0.21]",9,fragments,4,2020-02-28 11:20:37+00:00,POINT (340208.1528626664 6288395.023143975),43.603194,-1.457186,56.295898,ec501e35-b022-4c73-9988-a41218d6105e


In [12]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
target_store = 'postgre'
if target_store == 'postgre':
    # Get connection string information from env variables
    pg_conn_string = get_pg_connection_string()
    pg_connection = open_pg_connection(pg_conn_string)
    pg_cursor = pg_connection.cursor()
    # Store row_id when insert
    row_id_list = []

    for i, row in tqdm(df_data.iterrows()):
        try:
            row_id = insert_trash_df(row, pg_cursor, pg_connection)
            logger.info(row_id)
            row_id_list.append(row_id)
        except:
            prediction_id = row['id']
            logger.error(
                f'There was an issue inserting Trash id: {prediction_id} within PostGre')
            logger.error(
                "Early exit of ETL workflow as PG INSERT failed")
            exit()

    logger.info(
        f'Successfully inserted {str(len(row_id_list))} Trashes within Trash table')

    # Close PG connection
    close_pg_connection(pg_connection)

INFO:root:Connection established
0it [00:00, ?it/s]INFO:root:d5ff2788-5629-41bf-b048-2dcaa5918fca
1it [00:00,  1.15it/s]INFO:root:6dd6c877-f9ab-4a29-b255-df79eb9ea460
2it [00:01,  1.38it/s]INFO:root:8e22febf-853b-4473-a9fc-a0aead2854b5
3it [00:01,  1.61it/s]INFO:root:6fd37e72-cd52-498c-9bf1-eb2d43c4fd66
4it [00:02,  1.80it/s]INFO:root:78239425-de43-442c-8a43-4208d73586d3
5it [00:02,  1.99it/s]INFO:root:0ee36eda-0e6b-4069-aed9-48b8c0103a19
6it [00:02,  2.15it/s]INFO:root:e19e4361-c26b-4969-b1ed-c67b7788a9bc
7it [00:03,  2.28it/s]INFO:root:ae88887d-b1ae-4f15-991d-bbc8726a14b9
8it [00:03,  2.34it/s]INFO:root:bcefcf8f-04d5-419a-8772-4da5cd9aebbc
9it [00:03,  2.36it/s]INFO:root:68c0c677-f5db-46a6-8e2e-a16e7b2097ab
10it [00:04,  2.44it/s]INFO:root:493fafa0-e448-47ef-b6a6-333be2f08c2f
11it [00:04,  2.36it/s]INFO:root:f0c6d502-084b-4d9d-8c0c-db7cbc918d34
12it [00:05,  2.43it/s]INFO:root:b7be1668-589a-41e3-8032-f13ea0333e14
13it [00:05,  2.47it/s]INFO:root:3b916048-5251-47b9-82fa-76693ca642b0
1